<a href="https://colab.research.google.com/github/smiling2727/-Example-Text-To-Video-AI/blob/main/NovelTranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
input_novel_drive_path = '/content/drive/MyDrive/video_generation/novel_translate/input' # @param {type:"string"}
output_novel_drive_path = '/content/drive/MyDrive/video_generation/novel_translate/output' # @param {type:"string"}
input_txt_filename='buerzhichen.txt' # @param {type:"string"}
novel_name='buerzhichen' # @param {type:"string"}

In [5]:
# install packages
!pip install tqdm
!pip install anthropic
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 6.7 MB/s eta 0:00:00


In [29]:
from google.colab import userdata
import anthropic
from openai import OpenAI
# 点击colab做吧的钥匙(Secrets)按钮，并将CLAUDE_API_KEY加入列表中，勾选Notebook access
CLAUDE_API_KEY = userdata.get("CLAUDE_API_KEY")
client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
open_ai_client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [ ]:
import os
import re
import json
import anthropic
from tqdm import tqdm

def read_input_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def sample_text(text, sample_size=5000, num_samples=30):
    length = len(text)
    samples = []
    for i in range(num_samples):
        start = (i * length) // num_samples
        end = start + sample_size
        samples.append(text[start:end])
    return samples

def extract_and_translate_terms(sample, previous_results=""):
    # 解析之前的结果，获取最后使用的编号
    last_number = 0
    existing_terms = {}
    for line in previous_results.split('\n'):
        if line.strip():
            match = re.match(r'(\d+)\.\s(.+?):', line)
            if match:
                number, term = match.groups()
                last_number = max(last_number, int(number))
                existing_terms[term] = int(number)

    prompt = f"""
    请从以下中文小说文本样本中提取并翻译以下类型的术语：
    1. 人物名字
    2. 家族名称
    3. 地名（星球、国家、城市等）
    4. 组织名称（军队、政府机构等）
    5. 特殊术语（与故事世界观相关的概念）

    重要规则：
    1. 保持之前翻译结果中的编号不变。
    2. 对于新的术语，从{last_number + 1}开始编号。
    3. 如果某个术语之前已经翻译过，在描述中添加"(之前已翻译)"，但仍然提供完整的英文翻译。
    4. 人名翻译：
       - 为每个中文姓氏选择一个固定的英文姓氏，并在整个家族中保持一致。
       - 例如，如果"颜"家族的一个成员被翻译为"Evelyn Yance"，那么所有颜家成员都应使用"Yance"作为姓氏。不要使用中文的拼音。
       - 个人名字应该英语化，但可以保留一些原名的特点，应当改为英文名字，避免使用拼音。

    5. 家族名称：
       - 使用选定的英文姓氏+"family"。例如："颜家" -> "Yance family"

    6. 地名和组织名称：
       - 可以适当音译或意译，但要保持其独特性和科幻感。
       - 对于虚构的地名，可以创造性地翻译，但要确保其含义或特点得到保留。

    7. 特殊术语：
       - 应根据其含义进行翻译，同时注重保持科幻感和独特性。
       - 对于关键概念（如"alpha"、"omega"、"信息素"等），保持一致的翻译。

    8. 翻译风格一致性：
       - 对于人名，要么全部英语化，要么在英语化的基础上保留一些原始发音的特点，但要在整个文本中保持一致。
       - 例如，如果决定将"冯"翻译为"Felix"，那么其他类似的中文名字也应该采用类似的英语化方法。

    9. 保持所有翻译的一致性：
       - 如果某个术语之前已经翻译过，请在描述中添加"(之前已翻译)"，但仍然提供完整的英文翻译。
       - 在描述中注明"之前已翻译"，以便跟踪和维护一致性。

    请以以下格式输出结果：
    编号. 中文术语 (English Translation): 简短描述和类型（人名/地名/组织/术语等）

    之前的翻译结果：
    {previous_results}

    新文本样本：
    {sample}
    """

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=8000,
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

def process_samples(samples):
    all_results = ""
    term_mapping = {}
    for i, sample in enumerate(tqdm(samples, desc="Processing samples")):
        results = extract_and_translate_terms(sample, all_results)
        all_results += results + "\n\n"

        # 解析结果并更新术语映射
        for line in results.split('\n'):
            if ':' in line:
                number_and_term, description = line.split(':', 1)
                match = re.match(r'(\d+)\.\s(.+?)\s*\(([^)]+)\)', number_and_term)
                if match:
                    number, chinese_term, english_term = match.groups()
                    term_mapping[chinese_term] = {
                        "number": number,
                        "english_term": english_term.strip(),
                        "description": description.strip()
                    }

    return term_mapping

def save_term_mapping(term_mapping, novel_name, output_novel_drive_path):
    output_file = f"Name_mapping_{novel_name}.json"
    # output_path = os.path.join(output_novel_drive_path, output_file)
    output_novel_name_path = os.path.join(output_novel_drive_path, novel_name)
    os.makedirs(output_novel_name_path, exist_ok=True)
    output_path = os.path.join(output_novel_name_path, output_file)
    print(f"Saving term mapping to {output_path}")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(term_mapping, f, ensure_ascii=False, indent=4)
    print(f"Term mapping saved to {output_path}")


def step_1_term_mapping_main(input_novel_drive_path, input_txt_filename, output_novel_drive_path, novel_name):
    input_file = os.path.join(input_novel_drive_path, input_txt_filename)
    print(f"Input file path: {input_file}")
    print("Reading input file...")
    chinese_text = read_input_file(input_file)
    print("Sampling text...")
    samples = sample_text(chinese_text)
    print("Processing samples and generating term mapping...")
    term_mapping = process_samples(samples)
    print("Saving term mapping...")
    save_term_mapping(term_mapping, novel_name, output_novel_drive_path)
    print("Term mapping process completed.")

# def get_novel_name_mapping(input_novel_drive_path, input_txt_filename, output_novel_drive_path, novel_name):
#    main(input_novel_drive_path, input_txt_filename, output_novel_drive_path, novel_name)

step_1_term_mapping_main(input_novel_drive_path, input_txt_filename, output_novel_drive_path, novel_name)

Input file path: /content/drive/MyDrive/video_generation/novel_translate/input/buerzhichen.txt
Reading input file...
Sampling text...
Processing samples and generating term mapping...


Processing samples: 100%|██████████| 30/30 [03:09<00:00,  6.31s/it]

Saving term mapping...
Saving term mapping to /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/Name_mapping_buerzhichen.json
Term mapping saved to /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/Name_mapping_buerzhichen.json
Term mapping process completed.


In [ ]:
import json
import anthropic
import os

def read_json_file(output_novel_drive_path, novel_name, file_name):
    """从小说专属文件夹读取JSON文件"""
    novel_folder = os.path.join(output_novel_drive_path, novel_name)
    print(f"novel_folder = {novel_folder}")
    file_path = os.path.join(novel_folder, file_name)
    print(f"novel_file_path = {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"在 {novel_folder} 中找不到文件 '{file_name}'。")
        return None
    except json.JSONDecodeError:
        print(f"'{file_name}' 不是有效的JSON文件。")
        return None

def write_json_file(output_novel_drive_path, content, novel_name, file_name):
    """将JSON内容写入小说专属文件夹"""
    novel_folder = os.path.join(output_novel_drive_path, novel_name)
    # 确保文件夹存在
    if not os.path.exists(novel_folder):
        os.makedirs(novel_folder)
    file_path = os.path.join(novel_folder, file_name)
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(content, f, ensure_ascii=False, indent=4)
        print(f"文件已成功保存为 '{file_path}'。")
    except IOError:
        print(f"无法写入文件 '{file_path}'。")

def clean_json_response(response_text):
    """清理和修复JSON响应中的常见问题"""
    try:
        # 移除可能的前缀和后缀文本
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        if json_start != -1 and json_end != -1:
            response_text = response_text[json_start:json_end]

        # 移除最后一项后的逗号
        response_text = re.sub(r',(\s*})', r'\1', response_text)

        # 确保所有字符串使用双引号
        response_text = re.sub(r"'([^']*)':", r'"\1":', response_text)
        response_text = re.sub(r':\s*\'([^\']*)\'', r':"\1"', response_text)

        return response_text
    except Exception as e:
        print(f"清理JSON时出错: {str(e)}")
        return response_text

def validate_term_mapping(terms):
    """验证术语映射的格式和内容"""
    try:
        if not isinstance(terms, dict):
            raise ValueError("术语映射必须是一个字典")

        for term, info in terms.items():
            # 验证基本结构
            if not isinstance(info, dict):
                raise ValueError(f"术语 '{term}' 的信息必须是一个字典")

            # 验证必需字段
            required_fields = ['number', 'english_term', 'description']
            for field in required_fields:
                if field not in info:
                    raise ValueError(f"术语 '{term}' 缺少必需字段 '{field}'")

            # 验证number格式
            if not str(info['number']).isdigit():
                raise ValueError(f"术语 '{term}' 的编号必须是数字")

            # 验证字段类型
            if not isinstance(info['english_term'], str):
                raise ValueError(f"术语 '{term}' 的english_term必须是字符串")
            if not isinstance(info['description'], str):
                raise ValueError(f"术语 '{term}' 的description必须是字符串")

        return True
    except Exception as e:
        print(f"验证失败: {str(e)}")
        return False

def process_terms_with_claude(terms, output_novel_drive_path, novel_name):
    terms_str = json.dumps(terms, ensure_ascii=False, indent=2)
    prompt = f"""
    请处理以下术语列表，遵循这些规则：
    1. 按照英文术语（english_term）的字母顺序对所有条目进行排序。
    2. 从1开始连续重新编号，更新"number"字段值。
    3. 严格注意：对于人名，确保使用完全英语的名字，可以对json进行更改。必须必须避免中文名字的拼音，而是根据人物的特点和身份起一个新的英文名字和英文姓氏，并遵循英文姓名顺序（名在前，姓在后）。
    4. 检查并改进所有翻译，确保准确性和一致性。
    5. 保持每个术语的中文名（键名）不变。
    6. 输出格式必须是有效的JSON，确保：
       - 所有字符串都用双引号（"）括起来
       - 所有字符串都正确结束，不包含未转义的引号
       - 所有括号都正确配对
       - 所有值都用逗号正确分隔，最后一项不要有逗号
    7. 确保所有原有的条目都包含在输出中，不要遗漏或添加额外的条目。
    8. 直接返回JSON对象，不要添加任何额外的解释或说明。

    原始术语列表：
    {terms_str}
    """
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            # model="claude-3-opus-latest",
            max_tokens=8000,
            temperature=0.2,
            messages=[{"role": "user", "content": prompt}]
        )

        response_text = response.content[0].text.strip()

        # 清理JSON响应
        cleaned_text = clean_json_response(response_text)

        try:
            # 解析JSON
            processed_terms = json.loads(cleaned_text)

            # 验证处理后的数据
            if validate_term_mapping(processed_terms):
                return processed_terms
            else:
                raise ValueError("术语映射验证失败")

        except (json.JSONDecodeError, ValueError) as e:
            print(f"JSON处理错误: {str(e)}")

            # 保存调试信息
            debug_dir = os.path.join(os.path.dirname(os.path.abspath(output_novel_drive_path)), novel_name, "debug")
            os.makedirs(debug_dir, exist_ok=True)

            # 保存原始响应、清理后的响应和错误信息
            with open(os.path.join(debug_dir, "raw_response.txt"), 'w', encoding='utf-8') as f:
                f.write(response.content[0].text)

            with open(os.path.join(debug_dir, "cleaned_response.txt"), 'w', encoding='utf-8') as f:
                f.write(cleaned_text)

            with open(os.path.join(debug_dir, "error_info.txt"), 'w', encoding='utf-8') as f:
                f.write(f"错误类型: {type(e).__name__}\n错误信息: {str(e)}")

            print(f"调试文件已保存到 {debug_dir} 文件夹")
            return None

    except Exception as e:
        print(f"处理术语时出现问题: {str(e)}")
        return None

def process_terms_with_gpt(terms, output_novel_drive_path, novel_name):
    terms_str = json.dumps(terms, ensure_ascii=False, indent=2)
    prompt = f"""
    请处理以下术语列表，遵循这些规则：
    1. 按照英文术语（english_term）的字母顺序对所有条目进行排序。
    2. 从1开始连续重新编号，更新"number"字段值。
    3. 严格注意：对于人名，确保使用完全英语的名字，可以对json进行更改。必须必须避免中文名字的拼音，而是根据人物的特点和身份起一个新的英文名字和英文姓氏，并遵循英文姓名顺序（名在前，姓在后）。
    4. 检查并改进所有翻译，确保准确性和一致性。
    5. 保持每个术语的中文名（key）不变。并且输出格式跟原始术语列表格式一致。 例子 "古董包": {{"number": "36", "english_term": "Vintage Bag", "description": "特殊术语,指具有收藏价值的旧包"}}
    6. 如果对翻译进行了改进，请在描述（description）字段末尾添加改进原因，格式为"(改进原因: [原因])"。
    7. 输出格式必须是有效的JSON，确保：
       - 所有字符串都用双引号（"）括起来
       - 所有字符串都正确结束，不包含未转义的引号
       - 所有括号都正确配对
       - 所有值都用逗号正确分隔，最后一项不要有逗号
    8. 确保所有原有的都必须包含在输出中，不要遗漏或添加额外的条目。
    9. 直接返回JSON对象，不要添加任何额外的解释或说明。

    原始术语列表：
    {terms_str}
    """
    try:
        # response = client.messages.create(
        #     model="claude-3-5-sonnet-20240620",
        #     # model="claude-3-opus-latest",
        #     max_tokens=8000,
        #     temperature=0.2,
        #     messages=[{"role": "user", "content": prompt}]
        # )

        completion = open_ai_client.chat.completions.create(
          model="gpt-4o",
          response_format={"type": "json_object" },
          messages=[
            {"role": "user", "content": prompt},
          ],
        )

        response_text = completion.choices[0].message.content

        # 清理JSON响应
        cleaned_text = clean_json_response(response_text)

        try:
            # 解析JSON
            processed_terms = json.loads(cleaned_text)

            # 验证处理后的数据
            if validate_term_mapping(processed_terms):
                return processed_terms
            else:
                raise ValueError("术语映射验证失败")

        except (json.JSONDecodeError, ValueError) as e:
            print(f"JSON处理错误: {str(e)}")

            # 保存调试信息
            debug_dir = os.path.join(os.path.dirname(os.path.abspath(output_novel_drive_path)), novel_name, "debug")
            os.makedirs(debug_dir, exist_ok=True)

            # 保存原始响应、清理后的响应和错误信息
            with open(os.path.join(debug_dir, "raw_response.txt"), 'w', encoding='utf-8') as f:
                f.write(response.content[0].text)

            with open(os.path.join(debug_dir, "cleaned_response.txt"), 'w', encoding='utf-8') as f:
                f.write(cleaned_text)

            with open(os.path.join(debug_dir, "error_info.txt"), 'w', encoding='utf-8') as f:
                f.write(f"错误类型: {type(e).__name__}\n错误信息: {str(e)}")

            print(f"调试文件已保存到 {debug_dir} 文件夹")
            return None

    except Exception as e:
        print(f"处理术语时出现问题: {str(e)}")
        return None

def step_2_mapping_cleanup_main(output_novel_drive_path, novel_name):
    input_file = f"Name_mapping_{novel_name}.json"
    output_file = f"Processed_mapping_{novel_name}.json"

    print(f"正在从 {novel_name} 文件夹读取 {input_file}...")
    terms = read_json_file(output_novel_drive_path, novel_name, input_file)

    if terms:
        print(f"读取到 {len(terms)} 个术语条目")
        # print("使用 Claude 处理术语...")
        # processed_terms = process_terms_with_claude(terms, output_novel_drive_path, novel_name)
        print("使用 GPT 处理术语...")
        processed_terms = process_terms_with_gpt(terms, output_novel_drive_path, novel_name)
        if processed_terms:
            print(f"成功处理 {len(processed_terms)} 个术语条目")
            print("写入输出文件...")
            write_json_file(output_novel_drive_path, processed_terms, novel_name, output_file)
        else:
            print("无法处理术语。请检查调试文件夹中的响应内容。")
    else:
        print("无法处理输入文件。请确保文件存在且格式正确。")
    print("处理完成。")


step_2_mapping_cleanup_main(output_novel_drive_path, novel_name)

正在从 buerzhichen 文件夹读取 Name_mapping_buerzhichen.json...
novel_folder = /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen
novel_file_path = /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/Name_mapping_buerzhichen.json
读取到 174 个术语条目
使用 GPT 处理术语...
成功处理 34 个术语条目
写入输出文件...
文件已成功保存为 '/content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/Processed_mapping_buerzhichen.json'。
处理完成。


In [ ]:
import json
import os
import re
import anthropic
import time
from tqdm import tqdm
import openai
from openai import RateLimitError, APIError

TRANSLATION_FOLDER = "translation"
def read_file(file_path):
    """从小说专属文件夹读取文件"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"找不到文件 '{file_path}'")
        return None

def write_chapter_file(output_novel_drive_path, content, chapter_number, novel_name):
    """将翻译后的章节保存到小说专属文件夹的Translated_Novelname子文件夹中"""
    novel_folder = os.path.join(output_novel_drive_path, novel_name)
    translation_folder = os.path.join(novel_folder, TRANSLATION_FOLDER)
    print("translation_folder", translation_folder)
    print('translation_folder: ', translation_folder)
    # 确保翻译文件夹存在
    if not os.path.exists(translation_folder):
        os.makedirs(translation_folder)
        print(f"创建翻译文件夹: {translation_folder}")

    file_name = f"chapter_{chapter_number:03d}.txt"
    file_path = os.path.join(translation_folder, file_name)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"已保存: {file_name}")

def load_term_mapping(output_novel_drive_path, novel_name):
    """从小说专属文件夹加载术语映射"""
    novel_folder = os.path.join(output_novel_drive_path, novel_name)
    file_name = f"Processed_mapping_{novel_name}.json"
    file_path = os.path.join(novel_folder, file_name)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"在 {novel_folder} 中找不到文件 '{file_name}'")
        return None

def split_by_chapters(text):
    chapter_pattern = re.compile(r'^第\d+章', re.MULTILINE)
    chapter_starts = [m.start() for m in chapter_pattern.finditer(text)]
    chapters = []
    for i in range(len(chapter_starts)):
        start = chapter_starts[i]
        end = chapter_starts[i+1] if i+1 < len(chapter_starts) else None
        chapter_content = text[start:end]
        chapters.append(chapter_content.strip())
    return chapters

def process_text(text):
    first_chapter_start = re.search(r'^第\d+章', text, re.MULTILINE)
    if first_chapter_start:
        prefix = text[:first_chapter_start.start()].strip()
        main_content = text[first_chapter_start.start():]
        chapters = split_by_chapters(main_content)
        if prefix:
            chapters.insert(0, prefix)
    else:
        chapters = [text]
    return chapters


def translate_chapter_by_gpt(chapter, term_mapping, open_ai_client, max_retries=3):
    term_mapping_str = json.dumps(term_mapping, ensure_ascii=False, indent=2)
    prompt = f"""
    Please translate the following Chinese text into English. Use the provided term mapping to ensure consistency of proper nouns.
    Maintain the original paragraph structure and ensure accuracy and fluency in the translation.

    Term Mapping:
    {term_mapping_str}

    Chinese Text:
    {chapter}

    Please provide the English translation. Provide only the translated text in English without adding any additional content or commentary. Ensure that no information is omitted from the original text.
    """

    for retry_group in range(2):  # Two groups of retries
        for attempt in range(max_retries):
            try:
                # Send the API request
                response = open_ai_client.chat.completions.create(
                    model="gpt-4o-mini",  # Replace with your model name
                    messages=[
                        {"role": "user", "content": prompt},
                    ],
                )

                # Debugging: Print the response structure
                print("Response:", response)

                # Extract the content from the response object
                response_text = response.choices[0].message.content
                return response_text

            except RateLimitError:
                print(f"Rate limit error encountered. Retrying... (Group {retry_group + 1}, Attempt {attempt + 1}/{max_retries})")
            except APIError as e:
                print(f"API error: {str(e)}")
                return None
            except Exception as e:
                print(f"Error during translation: {str(e)}")
                return None

        if retry_group == 0:
            print("First 3 attempts failed, waiting 30 seconds before next group...")
            time.sleep(10)  # Wait 30 seconds between retry groups

    print(f"Translation failed after {max_retries * 2} attempts.")
    return None



def translate_chapter(chapter, term_mapping, client, max_retries=3):
    term_mapping_str = json.dumps(term_mapping, ensure_ascii=False, indent=2)
    prompt = f"""
    请将以下中文文本翻译成英文。使用提供的术语映射来确保专有名词的一致性。
    保持原文的段落结构，并确保翻译的准确性和流畅性。

    术语映射：
    {term_mapping_str}

    中文文本：
    {chapter}

    Translate the Chinese text to English.  Provide only the translated text in English without adding any additional content or commentary. Ensure that no information is omitted from the original text.
    """

    for retry_group in range(2):  # 两组尝试
        for attempt in range(max_retries):
            try:
                response = client.messages.create(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=8000,
                    temperature=0.2,
                    messages=[{"role": "user", "content": prompt}]
                )
                return response.content[0].text
            except anthropic.APIError as e:
                if e.status_code == 500:
                    print(f"遇到服务器错误，正在重试... (组 {retry_group + 1}, 尝试 {attempt + 1}/{max_retries})")
                    if attempt < max_retries - 1:
                        time.sleep(5)  # 每次尝试之间等待5秒
                else:
                    print(f"API错误: {str(e)}")
                    return None
            except Exception as e:
                print(f"翻译时出错: {str(e)}")
                return None

        if retry_group == 0:
            print("前3次尝试失败，等待30秒后将进行下一组尝试...")
            time.sleep(30)  # 在两组尝试之间等待30秒

    print(f"在 {max_retries * 2} 次尝试后翻译失败")
    return None


def step_3_main(input_novel_drive_path, input_txt_filename, output_novel_drive_path, novel_name):
    # 使用与前面脚本相同的文件名格式
    # term_mapping_file = f"Processed_mapping_{novel_name}.json"
    input_file_path = os.path.join(input_novel_drive_path, input_txt_filename)
    print("input_novel_drive_path: ", input_novel_drive_path)
    print(f"Input file path: {input_file_path}")
    print("读取原文...")

    original_text = read_file(input_file_path)
    if not original_text:
        print("无法读取原文文件，程序终止。")
        return

    print("加载术语映射...")
    term_mapping = load_term_mapping(output_novel_drive_path, novel_name)
    if not term_mapping:
        print("无法加载术语映射文件，程序终止。")
        return

    print("分割章节...")
    chapters = process_text(original_text)
    print(f"总共分割出 {len(chapters)} 章")

    # 输出文件夹路径
    # current_dir = os.path.dirname(os.path.abspath(__file__))
    output_folder = os.path.join(output_novel_drive_path, novel_name, TRANSLATION_FOLDER)
    os.makedirs(output_folder, exist_ok=True) # create TRANSLATION_FOLDER
    print(f"翻译文件将保存到: {output_folder}")

    for i, chapter in enumerate(tqdm(chapters, desc="翻译进度")):
        print(f"\n翻译第 {i+1} 章...")
        translated_chapter = translate_chapter_by_gpt(chapter, term_mapping, open_ai_client)
        # translated_chapter = translate_chapter(chapter, term_mapping, client)
        if translated_chapter:
            # write_chapter_file(output_novel_drive_path, content, chapter_number, novel_name):
            write_chapter_file(output_novel_drive_path, translated_chapter, i+1, novel_name)
        else:
            print(f"第 {i+1} 章翻译失败，跳过")

    print(f"翻译完成！所有章节已保存到 {novel_name}/{TRANSLATION_FOLDER} 文件夹")


step_3_main(input_novel_drive_path, input_txt_filename, output_novel_drive_path, novel_name )

input_novel_drive_path:  /content/drive/MyDrive/video_generation/novel_translate/input
Input file path: /content/drive/MyDrive/video_generation/novel_translate/input/buerzhichen.txt
读取原文...
加载术语映射...
分割章节...
总共分割出 89 章
翻译文件将保存到: /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/translation


翻译进度:   0%|          | 0/89 [00:00<?, ?it/s]


翻译第 1 章...


翻译进度:   1%|          | 1/89 [00:00<00:58,  1.51it/s]

Response: ChatCompletion(id='chatcmpl-AX3YrujBG3HajBMcgLFhAnEGIKy7j', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"The Unsurpassable Minister"', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732441877, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0705bf87c0', usage=CompletionUsage(completion_tokens=7, prompt_tokens=7213, total_tokens=7220, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
translation_folder /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/translation
translation_folder:  /content/drive/MyDrive/video_generation/novel_translate/output/buerzhichen/translation
已保存: chapter_001.txt

翻译第 2 章...
